In [6]:
import os
import json
import base64
import numpy as np
from PIL import Image
from io import BytesIO
import zlib
import random
import shutil

Supervisely data uses JSON file with Bitmaps for each class title for their segmentations.
This script reads the bitmap and converts it to masks based on the colors for the classes.
This outputs a *_color.png used for visualization of the masks,
              *_masks.png containing a class for each pixel.
              for each subdirectory (train, valid, test)

EXAMPLE:

"classTitle": "urban_land",
            "bitmap": {
                "data": "eJy9VWs4VPsaH7kVm5wKR47sziQdREkNw2a33fZIbmGSS1s0aB9jH8xyz7NjN4/J3VZtGXJnhlHMWI3l8qhEYRBGzbLQFtMeltsc1SScNT3P/nA+ni9nfVjvb73r/b+/93lv/ywvD1cNtYNqOBxOg/S9kw8OpzKHwykmKihimrkOVW9MqHi5+zpjcmdnp+Z1OYIhhXgfjwu4bxdeUXA41WKS01nfxFIUOXm78cyBzNATCrmDmoPwHjhW5RxeeubirjdKcwV1+u2qT9VrLrj27iijXMm/fzJS2AryU66fUGhRk+BscgTozR+mOHsPu0u0+PE9j4L3Z8MTPfzgn6gEIqj/PjbU3/j5/b3P0PbZKWo8dVJFDGTEQQte5mYSq4YNASqZneK0UnVLYZ4QfRYH+TGNG45ag/qlX3zEIObmuvbjWfskjTDGIzpnUwesxc5OLFuedPzoG+EmJvpGkhkz7tpVwJqOqAu5Q7xAvPONFfoiNA3ZqkVYub+OJEaB9Xij+GhCA97IKkjUsL8mPkpsZdlMV6pSzN88X2VOJbDlOoL8j82uBkUjbTBORA/esWxyWbhKMZd8S9IQ4KWxLnfHf/ZheDb7bz4YSNNuLtCz2B2CPxWipUO7X/27cqi/qptJiNZ40suWQ7+t3Zg9bdk58zRLN9PZ4FhD4lnsaCFDV4lWxnURV+yz2M0mevqs3XhQj3FZ7C4ievmMJoh5F7LgJIP0wRlKnmhIYp3FnLSllYx7oIPC9Xer7ayxauAagT1WLdzkzE/wma8vhTCHE6g/yvhMpN8OSVrdHLBDyN500nK8Usl4lI9tSMLCIlLpUSSKKuezxo4A9p0cudMV9qPj4rqKK2Dp3jqAOiCLqse+14ZEJCzOjzJho+avvrq8QCzcg5UAGwusRwZBnx9H+Ri7Pd+seWNvIuaRs4SxrWm1G178z29p/sZu/wgZ6KQzJv/Sk1yfuJPicqeipR5LpJ5JEdHbLWOjP5vOmLBfsvRtdKn9YL/0kirPU9cylicGpXbS3myakqxR/AhhCYr8xXa+v6e4FBE9L3nb0kqfCiKU2GkXtWN6B11qZQ6C69OMIaxhcp45zndon4LpfmKAKoeBE6GFttReCu3u21z0KPbWDDzZXRE7xZBpFIdBzkyjTpGsxfafvV9ZQE/8B1dbbLmB46TYv5ZaQOVNd4nf6zQPMeBg+tCPrS2sjPC1S4IIWr7ZNfHAMq80JQRoVo7x6+sPP9G1FKJeRnQ2Z0qvgscT2RjK+YIkWjLry8AwZhXRH4FZlalPVwvyPcVJTyInk6dPiE6XEbIBnvQq2U9wJ9hvq0ozf8KChvu/PVOB3az9bw88ZuXme4YhHB51MgBcbIJzhOgu1M0vHdz0ovffBOavRZLNVoDjKGMj8kODZsF5V+hKErgYRTYTFS+e9OsaP9xSEczvgHlcUx30xcYjqdafmgm5Ru19JKHBMdwRih3lBOFRxvwM1Z/eT7kf/DCSzCkDepPBzZqhgr4qZP3jyt81C51B3qLdzANExN82qdj8RCVDXQYIJ7SVVfR2RDAnFKyf73YwEQPaqFobeCpt7pg1yI4iN5JtxwiGmL9A7qm0XbYvORc/YKQZ4W8QKsYTglKwonXAG1xTIRup1Cz8JQHKxYh5gRNnW+4H80WBXGmq+3dQ7GkwUiy7mwnMr1PFW7g8uYzYlst2V4hkcORWAdCLoQAM/fwFORscUV8BOqVMOhkiMfF5TdhCKSDFpFrOGM4elxzgx3fha0Si8xKt13IUu2OYw+MML5ctwbJcdymQs+Yxr4K6rzg6WHEIBmXXxL0drBzjhQTb/7kDqNX+qv2s8vE9lqwn1bn53oYUWmaqBNs8N1OTF4HDmX7Mw/eWIYROdDrTw0fm2OrlWHBKYmC42Ty1ISz8pQOsTMt2V1kE5oRUH3F/mGlE+2hjzCjVW/V55Wu4SfJ1ABfLDH906T1QFvhwWwe1I5Rk0KA0PKV9NBW8h83Lbgf0qxiwuxviH5TP2DR6W7iaF5cIxtkZd8ABZfs/wVz5IC5qop8Fe6ZRkaNtd9zjh8wZj3YbrhQmlcNbFblFr673rXRHG2+BRI8rKVBetHF0wnxnwm8z2zxo/b00748gPdBfgNpfFtthi+9m9N42jPpf+/TbWvOioG0b30a4c/hdEXYWAaxfwfIVSYN7BNzgzb+ZCNCFD1vFAQbGDymBTz4AtzCjIeAoIZock3AreZj5avtFAmggHY8qhF1IETa+h9BD5cMA4CpdwY+0FkVD4l5hSZv+7b66aEi29oeIk369vQ1eHjlTi26nyuEIJ4gXB6Ghe1A3py9tzjXVRV84/xdMAuHpd6spV9OxGWN/Y3ivO1uIHkovYWM1yaFBfeULldhiiEE4tFQ9MT0B6llueofvKx/bqUenL9vpYVfpp3MGX5uCM7/47yhGV7kdCytIX5bXnuTs4cT57ocb/wF/d+nG",

Mapping for Colors and Classes

In [7]:
class_map = {
    "urban_land": 0,
    "agriculture_land": 1,
    "rangeland": 2,
    "forest_land": 3,
    "water": 4,
    "barren_land": 5,
    "unknown": 6
}
color_map = np.array([
    [0, 255, 255],     # urban_land - Cyan
    [255, 255, 0],     # agriculture_land - Yellow
    [255, 0, 255],     # rangeland - Magenta
    [0, 255, 0],       # forest_land - Green
    [0, 0, 255],       # water - Blue
    [255, 255, 255],   # barren_land - White
    [128, 128, 128]    # unknown - Gray
], dtype=np.uint8)

In [8]:
def decode_bitmap(data_str):
    raw = base64.b64decode(data_str)
    decompressed = zlib.decompress(raw)
    mask_img = Image.open(BytesIO(decompressed)).convert("L")
    return np.array(mask_img)

def convert_supervisely_to_mask(img_shape, objects):
    h, w = img_shape
    full_mask = np.zeros((h, w), dtype=np.uint8)
    for obj in objects:
        class_name = obj["classTitle"]
        if class_name not in class_map:
            continue
        class_id = class_map[class_name]
        bitmap = obj["bitmap"]
        origin_x, origin_y = bitmap["origin"]
        mask = decode_bitmap(bitmap["data"])
        binary_mask = (mask > 127)
        class_mask = np.zeros_like(mask, dtype=np.uint8)
        class_mask[binary_mask] = class_id
        region = full_mask[origin_y:origin_y+mask.shape[0], origin_x:origin_x+mask.shape[1]]
        region[binary_mask] = class_mask[binary_mask]
        full_mask[origin_y:origin_y+mask.shape[0], origin_x:origin_x+mask.shape[1]] = region
    return full_mask

In [4]:
os.listdir('deepglobe-land-cover-2018-DatasetNinja')

['LICENSE.md', 'meta.json', 'README.md', 'test', 'train', 'valid']

Converting the train dataset from DeepGlobe 2018 Dataset
    - Saves all output temporarily in DeepGlobe_Converted_Dataset/full/
    - Only the train folder in the original dataset has bitmaps, so only train/ is used for conversion

In [11]:
input_base = "deepglobe-land-cover-2018-DatasetNinja"
input_img_dir = os.path.join(input_base, "train", "img")
input_ann_dir = os.path.join(input_base, "train", "ann")
output_base = "DeepGlobe_Converted_Dataset"
temp_dir = os.path.join(output_base, "full")
os.makedirs(temp_dir, exist_ok=True)

for file in os.listdir(input_ann_dir):
    if file.endswith(".json"):
        name = file.replace("_sat.jpg.json", "")
        img_path = os.path.join(input_img_dir, f"{name}_sat.jpg")

        if not os.path.exists(img_path):
            print(f"Image file not found for annotation {file}")
            continue

        img = Image.open(img_path)
        width, height = img.size
        ann_path = os.path.join(input_ann_dir, file)

        with open(ann_path, "r") as f:
            ann = json.load(f)

        mask = convert_supervisely_to_mask((height, width), ann["objects"])

        img.save(os.path.join(temp_dir, f"{name}_sat.jpg"))
        Image.fromarray(mask).save(os.path.join(temp_dir, f"{name}_mask.png"))
        color_mask = color_map[mask]
        Image.fromarray(color_mask).save(os.path.join(temp_dir, f"{name}_color.png"))

print("Conversion complete.")

Conversion complete.


Splitting data 85/15 for training and validation
    - DeepGlobe_Converted_Dataset/train/
    - DeepGlobe_Converted_Dataset/valid/
    - Leaves /full/ empty

In [12]:
train_dir = os.path.join(output_base, "train")
valid_dir = os.path.join(output_base, "valid")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

triplets = sorted([f.replace("_sat.jpg", "") for f in os.listdir(temp_dir) if f.endswith("_sat.jpg")])
random.shuffle(triplets)
split_idx = int(0.85 * len(triplets))
train_ids = triplets[:split_idx]
valid_ids = triplets[split_idx:]

def move_triplet(name, dest):
    for suffix in ["_sat.jpg", "_mask.png", "_color.png"]:
        src = os.path.join(temp_dir, f"{name}{suffix}")
        dst = os.path.join(dest, f"{name}{suffix}")
        if os.path.exists(src):
            shutil.move(src, dst)

for name in train_ids:
    move_triplet(name, train_dir)

for name in valid_ids:
    move_triplet(name, valid_dir)

print(f"Done. {len(train_ids)} samples in train/, {len(valid_ids)} in valid/.")

Done. 682 samples in train/, 121 in valid/.


In [13]:
mask = np.array(Image.open("DeepGlobe_Converted_Dataset/train/119_mask.png"))
print("Unique values in mask:", np.unique(mask))
print(mask)

Unique values in mask: [0 1 2 4]
[[2 2 2 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


In [ ]:
print(len(os.listdir('data/train')))

892
